<h1>Quantitative Value Strategy</h1>

Value investing means investing stocks that are cheap compared to their intrinsic value (earnings, assets, etc...)

<h3>Importing Libraries</h3>

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy.stats import percentileofscore
import math

In [2]:
import warnings
warnings.filterwarnings('ignore')

<h3>Importing List of Stocks</h3>

In [3]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secret_keys import IEX_CLOUD_API_TOKEN

<h3>Creating a Data Frame to store our data</h3>

The value investing will be calculated with a combination of multiple metrics:
<ul>
    <li><b>Price-to-Earning Ratio: </b>The Stock price compared to the company's Earning</li>
    <li><b>Price-to-Book Ratio: </b>The Stock price compared to the percentage of ownership a stock gives</li>
    <li><b>Price-to-Sales Ratio: </b>The Stock price compared to the company's Revenue</li>
    <li><b>EV / EBITDA: </b>Entreprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization</li>
    <li><b>EV / GP: </b>Entreprise Value divided by Gross Profit</li>
</ul>

These metrics will be used to calculate a <b>Robust Value Score</b> which is the strategy we're using for this project

In [4]:
columns = [
    "Ticker",
    "Stock Price",
    "Price-to-Earnings Ratio",
    "Price-to-Earnings Percentile",
    "Price-to-Book Ratio",
    "Price-to-Book Percentile",
    "Price-to-Sales Ratio",
    "Price-to-Sales Percentile",
    "EV / EBITDA",
    "EV / EBITDA Percentile",
    "EV / GP",
    "EV / GP Percentile",
    "Robust Value Score",
    "Number of Shares to Buy"
]
df = pd.DataFrame(columns=columns)

<h3>Accessing the IEX Cloud API</h3>

The IEX Cloud API allows a batch request of 100 Tickers at a time. We need to split our list into multiple lists of 100 Tickers (or less). Here is a helper function that does this:

In [5]:
'''
Function that splits a list (l) into multiple lists of (n) items
args:
    l: array
    n: number
returns 2D array
'''
def split_list(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [6]:
# Split our symbol list into multiple list of 100 symbols
symbols = stocks["Ticker"]
symbol_lists = list(split_list(symbols, 100))

In order to execute a batch request with the IEX Cloud API, we need to enter the stocks in the URL separated by a comma.<br>

<b>Example</b>: [AAL, AAPL, ABBV] becomes "AAL,AAPL,AABBV"

In [7]:
symbol_batches = [] # the array that contains the symbols separated by commas
for symbol_list in symbol_lists:
    symbol_batches.append(",".join(symbol_list))

Get the key to access the API

In [8]:
from secret_keys import IEX_CLOUD_API_TOKEN

Do batch requests

In [9]:
for symbol_batch in symbol_batches:
    url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_batch}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(url).json()
    for symbol in symbol_batch.split(","):
        if symbol in data:
            latestPrice = data[symbol]["quote"]["latestPrice"]
            peRatio = data[symbol]["quote"]["peRatio"]
            priceToBook = data[symbol]["advanced-stats"]["priceToBook"]
            priceToSales = data[symbol]["advanced-stats"]["priceToSales"]
            enterpriseValue = data[symbol]["advanced-stats"]["enterpriseValue"]
            EBITDA = data[symbol]["advanced-stats"]["EBITDA"]
            grossProfit = data[symbol]["advanced-stats"]["grossProfit"]
            # Filter missing data
            if latestPrice != None and peRatio != None and priceToBook != None and priceToSales != None and enterpriseValue != None and EBITDA != None and grossProfit != None:
                # The percentiles are currently filled with N/A because we need all the data to calculate them
                df = df.append(pd.Series([
                    symbol,
                    latestPrice,
                    peRatio,
                    "N/A",
                    priceToBook,
                    "N/A",
                    priceToSales,
                    "N/A",
                    enterpriseValue / EBITDA,
                    "N/A",
                    enterpriseValue / grossProfit,
                    "N/A",
                    "N/A",
                    "N/A"
                ], index=columns), ignore_index=True )

In [10]:
df

,Ticker,Stock Price,Price-to-Earnings Ratio,Price-to-Earnings Percentile,Price-to-Book Ratio,Price-to-Book Percentile,Price-to-Sales Ratio,Price-to-Sales Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,Robust Value Score,Number of Shares to Buy
0,A,120.54,29.42,N/A,7.1600,N/A,5.560,N/A,21.385746,N/A,10.966113,N/A,N/A,N/A
1,AAL,13.80,-3.77,N/A,-1.0136,N/A,0.259,N/A,-20.852068,N/A,0.944496,N/A,N/A,N/A
2,AAP,184.84,20.61,N/A,3.8800,N/A,1.020,N/A,11.031209,N/A,2.472969,N/A,N/A,N/A
3,AAPL,143.99,22.66,N/A,35.6900,N/A,6.220,N/A,17.836217,N/A,14.109903,N/A,N/A,N/A
4,ABBV,153.20,22.07,N/A,16.8400,N/A,4.990,N/A,11.782571,N/A,8.571050,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,YUM,115.47,21.40,N/A,-3.9000,N/A,5.080,N/A,19.869742,N/A,9.213526,N/A,N/A,N/A
478,ZBH,107.64,109.40,N/A,1.9600,N/A,3.030,N/A,18.229777,N/A,5.371973,N/A,N/A,N/A
479,ZBRA,308.52,19.94,N/A,5.8000,N/A,2.860,N/A,15.511459,N/A,6.676221,N/A,N/A,N/A
480,ZION,52.99,5.68,N/A,1.3600,N/A,2.360,N/A,4.389846,N/A,2.200136,N/A,N/A,N/A


<h3>Calculating Percentile Columns</h3>

In [11]:
column_to_percentile_map = {
    "Price-to-Earnings Ratio": "Price-to-Earnings Percentile",
    "Price-to-Book Ratio": "Price-to-Book Percentile",
    "Price-to-Sales Ratio": "Price-to-Sales Percentile",
    "EV / EBITDA": "EV / EBITDA Percentile",
    "EV / GP": "EV / GP Percentile",
    
}

In [12]:
for col in column_to_percentile_map:
    for row in df.index:
        df.loc[row, column_to_percentile_map[col]] = percentileofscore(df[col], df.loc[row, col]) / 100

In [13]:
df

,Ticker,Stock Price,Price-to-Earnings Ratio,Price-to-Earnings Percentile,Price-to-Book Ratio,Price-to-Book Percentile,Price-to-Sales Ratio,Price-to-Sales Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,Robust Value Score,Number of Shares to Buy
0,A,120.54,29.42,0.726141,7.1600,0.809129,5.560,0.813278,21.385746,0.807054,10.966113,0.80083,N/A,N/A
1,AAL,13.80,-3.77,0.03527,-1.0136,0.062241,0.259,0.010373,-20.852068,0.008299,0.944496,0.037344,N/A,N/A
2,AAP,184.84,20.61,0.518672,3.8800,0.596473,1.020,0.159751,11.031209,0.375519,2.472969,0.155602,N/A,N/A
3,AAPL,143.99,22.66,0.56639,35.6900,0.96888,6.220,0.838174,17.836217,0.697095,14.109903,0.883817,N/A,N/A
4,ABBV,153.20,22.07,0.551867,16.8400,0.929461,4.990,0.76556,11.782571,0.414938,8.571050,0.661826,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,YUM,115.47,21.40,0.53527,-3.9000,0.058091,5.080,0.776971,19.869742,0.757261,9.213526,0.713693,N/A,N/A
478,ZBH,107.64,109.40,0.983402,1.9600,0.30083,3.030,0.559129,18.229777,0.711618,5.371973,0.414938,N/A,N/A
479,ZBRA,308.52,19.94,0.497925,5.8000,0.738589,2.860,0.532158,15.511459,0.616183,6.676221,0.520747,N/A,N/A
480,ZION,52.99,5.68,0.080913,1.3600,0.170124,2.360,0.445021,4.389846,0.072614,2.200136,0.136929,N/A,N/A


<h3>Calculating the Robust Value Score (RV Score)</h3>

In [14]:
from statistics import mean

for row in df.index:
    value_percentiles = []
    for col in column_to_percentile_map.keys():
        value_percentiles.append(df.loc[row, column_to_percentile_map[col]])
    df.loc[row, "Robust Value Score"] = mean(value_percentiles)

In [15]:
df

,Ticker,Stock Price,Price-to-Earnings Ratio,Price-to-Earnings Percentile,Price-to-Book Ratio,Price-to-Book Percentile,Price-to-Sales Ratio,Price-to-Sales Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,Robust Value Score,Number of Shares to Buy
0,A,120.54,29.42,0.726141,7.1600,0.809129,5.560,0.813278,21.385746,0.807054,10.966113,0.80083,0.791286,N/A
1,AAL,13.80,-3.77,0.03527,-1.0136,0.062241,0.259,0.010373,-20.852068,0.008299,0.944496,0.037344,0.030705,N/A
2,AAP,184.84,20.61,0.518672,3.8800,0.596473,1.020,0.159751,11.031209,0.375519,2.472969,0.155602,0.361203,N/A
3,AAPL,143.99,22.66,0.56639,35.6900,0.96888,6.220,0.838174,17.836217,0.697095,14.109903,0.883817,0.790871,N/A
4,ABBV,153.20,22.07,0.551867,16.8400,0.929461,4.990,0.76556,11.782571,0.414938,8.571050,0.661826,0.66473,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,YUM,115.47,21.40,0.53527,-3.9000,0.058091,5.080,0.776971,19.869742,0.757261,9.213526,0.713693,0.568257,N/A
478,ZBH,107.64,109.40,0.983402,1.9600,0.30083,3.030,0.559129,18.229777,0.711618,5.371973,0.414938,0.593983,N/A
479,ZBRA,308.52,19.94,0.497925,5.8000,0.738589,2.860,0.532158,15.511459,0.616183,6.676221,0.520747,0.58112,N/A
480,ZION,52.99,5.68,0.080913,1.3600,0.170124,2.360,0.445021,4.389846,0.072614,2.200136,0.136929,0.18112,N/A


<h3>Selecting the 50 Best Value Stocks</h3>

In [16]:
# Sort the values of the Data Frame
df.sort_values("Robust Value Score", ascending=True, inplace=True)

# Extract the 50 first elements of the sorted Data Frame
df = df[:50]

# Reset the indices
df.reset_index(drop=True, inplace=True)

In [17]:
df

,Ticker,Stock Price,Price-to-Earnings Ratio,Price-to-Earnings Percentile,Price-to-Book Ratio,Price-to-Book Percentile,Price-to-Sales Ratio,Price-to-Sales Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,Robust Value Score,Number of Shares to Buy
0,AAL,13.80,-3.77,0.03527,-1.0136,0.062241,0.2590,0.010373,-20.852068,0.008299,0.944496,0.037344,0.030705,N/A
1,AIG,54.22,4.76,0.062241,0.7777,0.080913,0.8067,0.122407,2.820521,0.03112,0.784786,0.018672,0.063071,N/A
2,GPS,9.29,-48.76,0.012448,1.4300,0.187759,0.2133,0.008299,4.835500,0.093361,0.725286,0.016598,0.063693,N/A
3,GM,34.76,5.71,0.085062,0.8312,0.085062,0.4053,0.03527,1.996040,0.020747,1.636862,0.095436,0.064315,N/A
4,PVH,61.52,4.47,0.060166,0.7970,0.082988,0.4694,0.047718,4.719181,0.082988,1.052998,0.049793,0.06473,N/A
5,LNC,48.96,7.27,0.116183,0.5723,0.068465,0.4377,0.041494,5.275176,0.109959,0.429325,0.002075,0.067635,N/A
6,KSS,37.60,5.69,0.082988,1.1400,0.136929,0.2628,0.012448,3.814013,0.047718,1.125477,0.060166,0.06805,N/A
7,UNM,35.86,7.74,0.124481,0.6725,0.074689,0.6023,0.076763,4.036981,0.058091,0.604232,0.008299,0.068465,N/A
8,F,12.03,4.24,0.056017,1.0600,0.120332,0.3646,0.029046,3.750555,0.045643,1.880443,0.107884,0.071784,N/A
9,SYF,29.01,3.92,0.051867,1.2000,0.145228,0.8912,0.130705,2.527605,0.026971,0.924486,0.033195,0.077593,N/A


<h3>Calculating the number of shares to buy</h3>

Get the user's portfolio size by prompting an input box

In [18]:
def get_portfolio_size():
    portfolio_size = input("Enter the value of your portfolio: ")
    
    try:
        val = float(portfolio_size)
    except ValueError:
        print("\nYou need to enter a number.")
        val = get_portfolio_size()
    
    return val

In [19]:
portfolio_size = get_portfolio_size()

Enter the value of your portfolio: 2500000


In [20]:
position_size = portfolio_size / len(df.index)

for row in df.index:
    df.loc[row, "Number of Shares to Buy"] = position_size // df.loc[row, "Stock Price"]

df

,Ticker,Stock Price,Price-to-Earnings Ratio,Price-to-Earnings Percentile,Price-to-Book Ratio,Price-to-Book Percentile,Price-to-Sales Ratio,Price-to-Sales Percentile,EV / EBITDA,EV / EBITDA Percentile,EV / GP,EV / GP Percentile,Robust Value Score,Number of Shares to Buy
0,AAL,13.80,-3.77,0.03527,-1.0136,0.062241,0.2590,0.010373,-20.852068,0.008299,0.944496,0.037344,0.030705,3623.0
1,AIG,54.22,4.76,0.062241,0.7777,0.080913,0.8067,0.122407,2.820521,0.03112,0.784786,0.018672,0.063071,922.0
2,GPS,9.29,-48.76,0.012448,1.4300,0.187759,0.2133,0.008299,4.835500,0.093361,0.725286,0.016598,0.063693,5382.0
3,GM,34.76,5.71,0.085062,0.8312,0.085062,0.4053,0.03527,1.996040,0.020747,1.636862,0.095436,0.064315,1438.0
4,PVH,61.52,4.47,0.060166,0.7970,0.082988,0.4694,0.047718,4.719181,0.082988,1.052998,0.049793,0.06473,812.0
5,LNC,48.96,7.27,0.116183,0.5723,0.068465,0.4377,0.041494,5.275176,0.109959,0.429325,0.002075,0.067635,1021.0
6,KSS,37.60,5.69,0.082988,1.1400,0.136929,0.2628,0.012448,3.814013,0.047718,1.125477,0.060166,0.06805,1329.0
7,UNM,35.86,7.74,0.124481,0.6725,0.074689,0.6023,0.076763,4.036981,0.058091,0.604232,0.008299,0.068465,1394.0
8,F,12.03,4.24,0.056017,1.0600,0.120332,0.3646,0.029046,3.750555,0.045643,1.880443,0.107884,0.071784,4156.0
9,SYF,29.01,3.92,0.051867,1.2000,0.145228,0.8912,0.130705,2.527605,0.026971,0.924486,0.033195,0.077593,1723.0


<h3>Export the Data Frame to an Excel File Traders can use</h3>

Initialize the XlsxWriter Object

In [21]:
writer = pd.ExcelWriter("quantitative_value_strategy.xlsx", engine="xlsxwriter")

Pass the Data Frame to the Writer Object

In [22]:
SHEET_NAME = "Recommended Trades"
df.to_excel(writer, SHEET_NAME, index=False)

Format the excel file with the following rules:
<ul>
    <li><b>Tickers:</b> String format</li>
    <li><b>Stock Prices:</b> \$XX.XX</li>
    <li><b>Percentages:</b> %X.X</li>
    <li><b>Number of Shares to Buy:</b> Integer</li>
</ul>

In [23]:
# Setup the styles
BACKGROUND_COLOR = "#0A0A23"
FONT_COLOR = "#FFFFFF"
BORDER_WIDTH = 1

STRING_FORMAT = writer.book.add_format({
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

PRICE_FORMAT = writer.book.add_format({
    "num_format": "$0.00",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

VALUE_FORMAT = writer.book.add_format({
    "num_format": "$#,##0.00",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

NUMBER_FORMAT = writer.book.add_format({
    "num_format": "0",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

PERCENT_FORMAT = writer.book.add_format({
    "num_format": "0.0%",
    "font_color": FONT_COLOR,
    "bg_color": BACKGROUND_COLOR,
    "border": BORDER_WIDTH
})

map_columns_to_format = {
    "A": [columns[0], STRING_FORMAT],
    "B": [columns[1], PRICE_FORMAT],
    "C": [columns[2], NUMBER_FORMAT],
    "D": [columns[3], PERCENT_FORMAT],
    "E": [columns[4], NUMBER_FORMAT],
    "F": [columns[5], PERCENT_FORMAT],
    "G": [columns[6], NUMBER_FORMAT],
    "H": [columns[7], PERCENT_FORMAT],
    "I": [columns[8], NUMBER_FORMAT],
    "J": [columns[9], PERCENT_FORMAT],
    "K": [columns[10], NUMBER_FORMAT],
    "L": [columns[11], PERCENT_FORMAT],
    "M": [columns[12], PERCENT_FORMAT],
    "N": [columns[13], NUMBER_FORMAT]
}

In [24]:
# Format Cells
for col in map_columns_to_format:
    title, cell_format = map_columns_to_format[col]
    
    # Format Header
    writer.sheets[SHEET_NAME].write(f"{col}1", title, STRING_FORMAT)
    
    # Format Body
    writer.sheets[SHEET_NAME].set_column(f"{col}:{col}", 22, cell_format)
writer.save()

Save the writer to the Excel File

In [25]:
writer.save()